# IOT Project EDA Rest Workflow Context Porting Example

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

: 

In [ ]:
#Initialize and Start Execution
from fire_notebook.output.workflowcontext import RestWorkflowContext
import sys

In [ ]:
# Read CSV using pandas
df = pd.read_csv("IoTEnergySolutionsEDA.csv")
development = 'local1'

In [ ]:
# Cast 'timestamp' to pandas datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extract time fields (hour, minute, second)
df['timestamp_hour'] = df['timestamp'].dt.hour
df['timestamp_minute'] = df['timestamp'].dt.minute
df['timestamp_second'] = df['timestamp'].dt.second
# Extract year and month
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month

# Create concatenated 'Time' column
df['Time'] = df['timestamp_hour'].astype(str) + ':' + df['timestamp_minute'].astype(str) + ':' + df['timestamp_second'].astype(str)

In [ ]:
print(df.info())

In [ ]:
#IOT had only one parameter passed from App to workflow/Notebook for filetering devices
#Additional parameter to be added to the App ---> selectParameter
#If development environment is local, initializing custom parameter to 'selectDevices'
#selectParameter_values = ['energy_generation', 'energy_consumption', 'voltage', 'current', 'power', 'frequency']
parameter_from_App = ['selectDevice', 'selectParameter']
if development == 'local':
    parameters_custom = parameter_from_App[0]+'='+'meter_2'+parameter_from_App[1]+'='+'energy_generation'
    parameters_list = ["placeholder", "http://localhost", "default_job_id", parameters_custom]
else:
    parameters_list = sys.argv

In [ ]:
#Creating an instance of the RestWorkflowContext class
restworkflowcontext = RestWorkflowContext(parameters=parameters_list)

In [ ]:
#Initializing the value of the parameter from the App using getParameters API
#custom_parameters = "param1=value1,param2=value2,param3=value3
device_id = restworkflowcontext.getParmeters(parameter_name=parameter_from_App[0], default="meter_2")
graph_param = restworkflowcontext.getParmeters(parameter_name=parameter_from_App[1], default="energy_generation")
print('parameter[0] = ',device_id, ',','parameter[1] = ',graph_param)

In [ ]:
# Print the first few rows to ensure data was read correctly
restworkflowcontext.outPandasDataframe(9,"IoT Energy DF", df, 10)

# ---- Data Visualizations ----

In [ ]:

import plotly.graph_objs as go
import plotly.io as pio
import plotly
import plotly.offline as py

# Histogram for all feature columns

In [ ]:
#Get input from user for the selectParameter from list of options
#selectParameter_values = ['energy_generation', 'energy_consumption', 'voltage', 'current', 'power', 'frequency']
graph_param = restworkflowcontext.getParmeters(parameter_name=parameter_from_App[1], default="energy_generation")
graph_param

In [ ]:
fig = plotly.offline.plot([go.Histogram(x=df[graph_param], nbinsx=30)], 
              output_type='div', include_plotlyjs=False)

histogram_to_plot = f'{fig}'
restworkflowcontext.outPlotly(id=9,title="Histogram of energy_generation", text = histogram_to_plot)


# Boxplot to anlayze outliers

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio
import plotly.offline as py

In [ ]:
#Get input from user for the selectParameter from list of options
#selectParameter_values = ['energy_generation', 'energy_consumption', 'voltage', 'current', 'power', 'frequency']
graph_param = restworkflowcontext.getParmeters(parameter_name=parameter_from_App[1], default="voltage")
graph_param

In [ ]:
fig = plotly.offline.plot([go.Box(y=df[graph_param], boxpoints='all', jitter=0.3, pointpos=-1.8)],
                 output_type='div',
                include_plotlyjs=False)

In [ ]:
boxplot_to_plot = f'{fig}'
restworkflowcontext.outPlotly(id=9,title="Boxplot of {graph_param}", text = boxplot_to_plot)

# Correlation Matrix

In [ ]:
# Plot correlation matrix
numeric_cols = ['energy_generation', 'energy_consumption', 'voltage', 'current', 'power', 'frequency']
correlation_matrix = df[numeric_cols].corr()
custom_colorscale = [[0.0, "grey"],[0.5, "orange"],[1.0, "red"]]
fig = plotly.offline.plot([go.Heatmap(z=correlation_matrix.values,
                     x=correlation_matrix.columns,
                     y=correlation_matrix.columns,
                     colorscale=custom_colorscale)],
                 output_type='div',
                include_plotlyjs=False)

In [ ]:
correlation_matrix = f'{fig}'
restworkflowcontext.outPlotly(id=9,title="Correlation matrix of energy_generation", text = correlation_matrix)

# Line plot for energy consumption for selected device

In [ ]:
# 4. Filtering based on device IDs
df_filtered = df[df['device_id']==device_id]
device_id
print(df_filtered.head(100))

In [ ]:
# Print filtered device IDs
print('Devices:')
print(df_filtered['device_id'].unique())
print(df['month'].unique())

In [ ]:
#Get input from user for the selectParameter from list of options
#selectParameter_values = ['energy_generation', 'energy_consumption', 'voltage', 'current', 'power', 'frequency']
selectParameter_values = ['energy_generation', 'energy_consumption', 'voltage', 'current', 'power']
graph_param = restworkflowcontext.getParmeters(parameter_name=parameter_from_App[1], default=selectParameter_values)
graph_param


In [ ]:
energy = plotly.offline.plot([go.Scatter(x=df_filtered['timestamp'], y=df_filtered[graph_param])],
                 output_type='div',
                include_plotlyjs=False)

In [ ]:
energy_plotly = f'{energy}'
restworkflowcontext.outPlotly(id=9,title="Line plot of {graph_param}", text = energy_plotly)

In [ ]:
from IPython.display import HTML
import plotly.graph_objs as go
import plotly

energy = plotly.offline.plot(
    [go.Scatter(x=df_filtered['timestamp'], y=df_filtered[graph_param])],
    output_type='div',
    include_plotlyjs=True  # Needed for Jupyter
)

HTML(energy)